In [29]:
import dlc_practical_prologue
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from tqdm import trange

In [107]:
train_input, train_target, train_classes, test_input, test_target, test_classes = dlc_practical_prologue.generate_pair_sets(1000)

In [92]:
# Weight-sharing "Siamese" LeNet
class Siamese(nn.Module):

    def __init__(self):
        super(Siamese, self).__init__()
        
        self.LeNet1 = nn.Sequential(
            nn.Conv2d(1,16,5),  # 16x10x10 (input is 1x14x14)
            nn.MaxPool2d(2),    # 16x5x5
            nn.ReLU(),
            nn.Conv2d(16,32,2), # 32x4x4
            nn.MaxPool2d(2),    # 32x2x2 (-> 1x128 before LeNet2)
            nn.ReLU()
        )
        self.LeNet2 = nn.Sequential(
            nn.Linear(128,64),  # 1x64
            nn.ReLU(),
            nn.Linear(64,32),   # 1x32
            nn.ReLU()
        )
        self.LeNet3 = nn.Sequential(
            nn.Linear(32,16),   # 1x16
            nn.Sigmoid(),
            nn.Linear(16,2)     # 1x2
        )
        
    def forward_bro(self, x):
        x = self.LeNet1(x)
        x = x.view(-1,1,128)
        x = self.LeNet2(x)
        return x
    
    def forward(self, x1, x2):
        x1 = self.forward_bro(x1)
        x2 = self.forward_bro(x2)
        x3 = x1 + x2
        x3 = self.LeNet3(x3)
        return x3

In [86]:
# control convolutions' dimensions
x1 = train_input.narrow(0,0,100)
x1 = x1[:,0].view(100,1,14,14)
weight1 = torch.empty(16,1,5,5).normal_()
bias1 = torch.empty(16).normal_()
x1 = F.conv2d(x1, weight1, bias1)

x1 = F.max_pool2d(x1, 2)
x1 = F.relu(x1)

weight2 = torch.empty(32,16,2,2).normal_()
bias2 = torch.empty(32).normal_()
x1 = F.conv2d(x1, weight2, bias2)

x1 = F.max_pool2d(x1, 2)
x1 = F.relu(x1)

x1 = x1.view(-1,1,128)
x1.shape

In [122]:
def train_model(model, train_input, train_target, batch_size, nb_epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr = 1e-1)

    for e in range(nb_epochs):
        acc_loss = 0
        for b in range(0, train_input.size(0), batch_size):
            imgs = train_input.narrow(0, b, batch_size)
            imgs1 = imgs[:,0].view(batch_size, 1, 14, 14)
            imgs2 = imgs[:,1].view(batch_size, 1, 14, 14)
            output = model(imgs1, imgs2).view(batch_size, -1)
            loss = criterion(output, train_target.narrow(0, b, batch_size))
            acc_loss += loss.item()
            model.zero_grad()
            loss.backward()
            optimizer.step()
        print(e, acc_loss)

In [167]:
model = Siamese()
train_model(model, train_input, train_target, 100, 250)

0 6.990972399711609
1 6.934654772281647
2 6.935028254985809
3 6.9333906173706055
4 6.9332228899002075
5 6.930941700935364
6 6.932613790035248
7 6.9301891922950745
8 6.929503798484802
9 6.929929435253143
10 6.927061855792999
11 6.926907956600189
12 6.929477691650391
13 6.927361786365509
14 6.925181806087494
15 6.924086272716522
16 6.926539182662964
17 6.958846092224121
18 6.918391168117523
19 6.919260382652283
20 6.917004346847534
21 6.910929620265961
22 6.918404281139374
23 6.910204291343689
24 6.934026539325714
25 6.9286311864852905
26 6.916999697685242
27 6.920688569545746
28 6.906443417072296
29 6.925009191036224
30 6.888521254062653
31 6.860827326774597
32 6.86810177564621
33 6.887848198413849
34 6.879729509353638
35 6.892661154270172
36 6.869325935840607
37 6.893350660800934
38 6.831924796104431
39 6.863025009632111
40 6.880612790584564
41 6.831042945384979
42 6.823961913585663
43 6.848291754722595
44 6.7955580949783325
45 6.816515803337097
46 6.837901949882507
47 6.67449557781219

In [153]:
def compute_nb_errors(model, input_data, target_data, batch_size):
    nb_errors = 0
    
    for b in range(0, input_data.size(0), batch_size):
            imgs = input_data.narrow(0, b, batch_size)
            target = target_data.narrow(0, b, batch_size)
            imgs1 = imgs[:,0].view(batch_size, 1, 14, 14)
            imgs2 = imgs[:,1].view(batch_size, 1, 14, 14)
            output = model(imgs1, imgs2).view(batch_size, -1)
            pred = output.max(1)[1]
            nb_errors += (pred-target).abs().sum().item()
    
    return nb_errors

In [168]:
train_errors = compute_nb_errors(model, train_input, train_target, 100)
test_errors = compute_nb_errors(model, test_input, test_target, 100)

In [169]:
train_errors/train_input.size(0)

0.0

In [170]:
test_errors/test_input.size(0)

0.516